# Using LLM (CodeLlama-7b-Instruct-hf) for finding errors with code, giving completion score & find thought process of students
## Reasons to use CodeLlama-7b-Instruct-hf:
## •out of 7b,13b,34b,70b models ideally the most accurate results would be given by 70b parameters model & 34b model gives the best mix of speed and accuracy
## •we use 7b model due to **computational resources limits** (20 gb GPU VRAM limit)

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

import transformers
import torch
import pandas as pd
import re
from transformers import pipeline

2024-02-02 18:43:19.676857: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 18:43:19.676952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 18:43:19.992864: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Initializing Model and Pipeline

In [2]:
model = "codellama/CodeLlama-7b-Instruct-hf"

# Initializing the pipeline with the specific model
pipe = pipeline("text-generation", model=model,
    torch_dtype=torch.float16,
    device_map="auto",)

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

## Making a function to convert the text file with timestamped code to a DataFrame with Relevant Details.
## •Aids in easier prompting
## •Each Row is one Timestamp

In [3]:
def parse_file_with_details_to_df(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    blocks = content.split('.. activecode::')
    timestamps, codes, languages, activecodes = [], [], [], []

    for block in blocks[1:]:  # Skip the first split as it's before the first activecode block
        # Extract timestamp
        timestamp_match = re.search(r':timestamp: ([\d-]+\s[\d:]+)', block)
        # Find the start of the code block
        code_start = block.find(':code:') + len(':code:')
        # Extract language
        language_match = re.search(r':language: (\w+)', block)
        # Extract the activecode block name
        activecode_match = re.search(r'(\w+)\n', block)

        if timestamp_match and code_start > len(':code:'):
            timestamp = timestamp_match.group(1)
            code = block[code_start:].strip()  # Extract code, stripping leading/trailing whitespace
            language = language_match.group(1) if language_match else "Unknown"
            activecode = activecode_match.group(1) if activecode_match else "N/A"

            timestamps.append(timestamp)
            codes.append(code)
            languages.append(language)
            activecodes.append(activecode)

    # Create a DataFrame with the extracted details
    df = pd.DataFrame({
        'Timestamp': pd.to_datetime(timestamps),
        'Language': languages,
        'ActiveCode': activecodes,
        'Code': codes
    })

    return df

# Use the function with the path to your file
file_path = '/kaggle/input/1096-txt/1096.txt'
df = parse_file_with_details_to_df(file_path)
df.head()

,Timestamp,Language,ActiveCode,Code
0,2020-05-04 03:21:50,Python,intopost,from pythonds.basic import Stack\n \n def ...
1,2020-05-04 03:38:06,Python,intopost,from pythonds.basic import Stack\n \n def ...
2,2020-05-04 03:38:06,Python,intopost,from pythonds.basic import Stack\n \n def ...
3,2020-05-04 03:38:57,Python,intopost,from pythonds.basic import Stack\n \n def ...
4,2020-05-04 03:39:47,Python,intopost,from pythonds.basic import Stack\n \n def ...


# THEME/QUESTION GUESSING (to give LLM idea on how to evaluate student codes)
## •Since the questions corresponding solutions are not available, I am using the model to guess the question based on the solution and then usimg it for evaluation
## •If the Questions were made available (since they are already present in a database), the report would be more robust

In [4]:
# Definining prompt to make question(Student solution)
prompt = '''<s>[INST]
{{ '''+ df['Code'][0] + '''
   find the question this code is trying to solve:

}} [/INST]'''

# Generating text/code based on the solution prompt
result = pipe(prompt, max_length=1000, return_full_text=False)

# Extracting and printing the generated question
theme = result[0]['generated_text']
print(theme)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


  This code is trying to solve the problem of converting an infix expression to a postfix expression.

Infix expressions are written with operators between the operands, such as "A + B * C". The postfix expression is written with the operands followed by the operators, such as "AB\*C+".

The code defines a function `infixToPostfix` that takes an infix expression as input and returns the corresponding postfix expression. It does this by using a stack to keep track of the operators and their precedence.

The function first splits the input expression into a list of tokens, and then iterates through the tokens, processing each one as follows:

* If the token is an operand (either a letter or a number), it is added to the postfix list.
* If the token is an opening parenthesis, it is pushed onto the stack.
* If the token is a closing parenthesis, it is popped from the stack and added to the postfix list until the opening parenthesis is reached.
* If the token is an operator, it is compared 

## Initializing 'History' : Passes last few inference outputs to the LLM as part of prompt to give idea about sequential codes
## Initializing 'flow' : Will be converted to a Report in .txt format

In [5]:
history_list = []

history = str()

flow = "CODE TOPIC : " + str(df['ActiveCode'][0])  + "\nQUESTION : " + theme + "\n \n"

# ERROR FINDING ON THE BASIS OF THE QUESTION
## •We evaluate the code and students performance
## •For now, we allow the model to give report in its own way since no report format is mentioned
## •Ideally, if a given format is decided on, we can fine-tune the LLM or use multi shot prompting to give outputs in desired format

In [6]:
for index, row in df.iterrows():
    timestamp = row ['Timestamp']
    Language = row ['Language']
    code = row['Code']
    timestep = str(index)
    
    if (len(history_list))>2:
        history = ' '.join(history_list[-2:])
    else :
        history = ' '.join(history_list)
    
    if index==0:
        prompt = '''<s><<SYS>>Help find any issues with the students thinking process \n<</SYS>>[INST] Given Question :''' + theme +'''Language:'''+ Language + '''Timestep :''' + timestep +'''\n''' + code + '''explain errors in the Student's new code trying to solve the given question and give him an overall score out of 100. I dont need corrected code:}} [/INST]'''
    else:
        prompt = '''<s><<SYS>>Help find any issues with the students thinking process \n<</SYS>>[INST] Given Question :''' + theme +'''Language:'''+ Language + '''Timestep :''' + timestep +''' here is a list of errors the student was previously facing at different given timestamps: }} [/INST] {{''' + history + '''}} </s><s>[INST] {{Does this new code resolve any of the previous issues :''' + code + '''explain errors in the Student's new code trying to solve the given question and give him an overall score out of 100. I dont need corrected code:}} [/INST]'''
    
    
    # Generating text/code based on the solution prompt
    result = pipe(prompt, max_length=5000, return_full_text=False, temperature = 0.7, top_p = 0.95, top_k = 250, do_sample = True)
    
    # Extracting and printing the generated code
    generated = result[0]['generated_text']
    print(generated)
    
    temporary_history = "Timestep :" + timestep + "\nIssues : \n" + generated
    flow = flow + "\n" + "Timestamp :" + str(timestamp)+ "\nCode : \n" + code + "\nIssues : \n" + generated
    
    history_list.append(temporary_history)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


  Overall, the student's code seems to be a good attempt at solving the problem. However, there are a few errors and areas for improvement that I would like to highlight.

1. Error in the `prec` dictionary: The student has defined the precedence of the operators incorrectly. The operators should be defined in decreasing order of their precedence. For example, the precedence of multiplication should be higher than that of addition.
2. Error in the `opStack.pop()` method: The student has used the `opStack.pop()` method in a few places, but the method does not return the top element of the stack. Instead, it removes the top element from the stack and returns `None`. To fix this, the student should use the `opStack.peek()` method to peek at the top element of the stack, and then use `opStack.pop()` to remove it.
3. Error in the `while` loop: The student has used a `while` loop to pop elements from the stack until the opening parenthesis is reached. However, this loop is not necessary, as t

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


  The Student's new code seems to be a significant improvement over the previous code, but there are still a few issues that need to be addressed.

1. The `prec` dictionary is still not correct. The student has defined the precedence of the operators incorrectly. The operators should be defined in decreasing order of their precedence. For example, the precedence of multiplication should be higher than that of addition.
2. The `while` loop in the `infixToPostfix` function is still not necessary. The student can simply use the `opStack.pop()` method to pop the opening parenthesis and add it to the postfix list.
3. The `for` loop in the `infixToPostfix` function is still not necessary. The student can simply use the `tokenList.split()` method to split the input expression into a list of tokens.

Overall, the Student's new code has made significant progress, but there are still a few issues that need to be addressed. I would give the Student an overall score of 80 out of 100.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

  Overall, the Student's new code has made some progress in resolving the previous issues, but there are still some errors that need to be resolved. Here's a breakdown of the errors and their explanations:

1. The code still does not handle the case where there are multiple operators with the same precedence and the same associativity. For example, in the expression "A * B + C * D", the `*` operator has the same precedence and associativity as the `+` operator, but it should be handled before the `+` operator.
2. The code still does not handle the case where there are nested parentheses. For example, in the expression "( A + B ) * C - ( D - E ) * ( F + G )", the parentheses are nested, and the code should handle them correctly.
3. The code still does not handle the case where there are parentheses and operators with the same precedence. For example, in the expression "5 * 3 ** ( 4 - 2 )", the parentheses have the same precedence as the `**` operator, and the code should handle them cor

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


  The Student's new code has resolved some of the previous issues, but there are still some errors that need to be resolved. Here's a breakdown of the errors and their explanations:

1. The code still does not handle the case where there are multiple operators with the same precedence and the same associativity. For example, in the expression "A * B + C * D", the `*` operator has the same precedence and associativity as the `+` operator, but it should be handled before the `+` operator.
2. The code still does not handle the case where there are nested parentheses. For example, in the expression "( A + B ) * C - ( D - E ) * ( F + G )", the parentheses are nested, and the code should handle them correctly.
3. The code still does not handle the case where there are parentheses and operators with the same precedence. For example, in the expression "5 * 3 ** ( 4 - 2 )", the parentheses have the same precedence as the `**` operator, and the code should handle them correctly.

Overall, the St

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


  The Student's new code resolves some of the previous issues, but there are still some errors that need to be resolved. Here's a breakdown of the errors and their explanations:

1. The code still does not handle the case where there are nested parentheses. For example, in the expression "( A + B ) * C - ( D - E ) * ( F + G )", the parentheses are nested, and the code should handle them correctly.
2. The code still does not handle the case where there are parentheses and operators with the same precedence. For example, in the expression "5 * 3 ** ( 4 - 2 )", the parentheses have the same precedence as the `**` operator, and the code should handle them correctly.

Overall, the Student's new code has made some progress in resolving the previous issues, but there are still some errors that need to be resolved. I would give the Student an overall score of 70 out of 100.
  The Student's new code resolves some of the previous issues, but there are still some errors that need to be resolved. 

# Making an automated Report

In [7]:
file_path = 'check_1096.txt'

# Writing the string to a .txt file
with open(file_path, 'w') as file:
    file.write(flow)